In [41]:
from feature.make_dataset import make_data
from feature.after_dataset import accumulate
from feature.moving_average import moving_average
import pandas as pd
from glob import glob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')


In [42]:
train_input_list = './data/train_input/*.csv'
train_target_list = './data/train_target/*.csv'

test_input_list = './data/test_input/*.csv'
test_target_list = './data/test_target/*.csv'

In [43]:
train, test, p_train, p_test = make_data(train_input_list, train_target_list, test_input_list, test_target_list)
train_rol = moving_average(train_input_list, 6)
test_rol = moving_average(test_input_list, 6)

In [44]:
train = train.reset_index()
test = test.reset_index()
train_rol = train_rol.reset_index()
test_rol = test_rol.reset_index()

In [45]:
accumulate(train, test, '누적분무량')
accumulate(train, test, '누적백색광량')
accumulate(train, test, '누적청색광량')
accumulate(train, test, '누적적색광량')
accumulate(train, test, '누적총광량')

In [46]:
train = pd.concat([train, train_rol], axis=1)
test = pd.concat([test, test_rol], axis=1)

In [47]:
drop_feature = ['시간당분무량', '시간당백색광량', '시간당적색광량', '시간당청색광량', '시간당총광량', 'index']
train = train.drop(drop_feature, axis=1)
test = test.drop(drop_feature, axis=1)


In [48]:
train = train.drop(['Case', 'obs_time'], axis=1)
test = test.drop(['Case', 'obs_time'], axis=1)

In [49]:
train

,DAT,내부온도관측치,내부습도관측치,co2관측치,ec관측치,일간누적분무량,일간누적백색광량,일간누적적색광량,일간누적청색광량,일간누적총광량,...,"('일간누적총광량', 14)","('일간누적총광량', 15)","('일간누적총광량', 16)","('일간누적총광량', 17)","('일간누적총광량', 18)","('일간누적총광량', 19)","('일간누적총광량', 20)","('일간누적총광량', 21)","('일간누적총광량', 22)","('일간누적총광량', 23)"
0,0,27.119167,75.737500,531.675000,1.391470,3961.27,255677.8594,13698.4344,0.0000,269376.2938,...,124369.525000,143601.0470,162832.5690,182064.091000,201295.613000,220527.135000,236681.795633,249631.202600,259375.355900,265914.255533
1,1,27.895000,75.931667,518.283333,1.383575,3980.56,256324.5263,13708.3632,0.0000,270032.8895,...,124678.935000,143910.4570,163141.9790,182373.501000,201605.023000,220836.545000,237049.069917,250056.341167,259858.358750,266455.122667
2,2,26.640000,75.680833,537.283333,1.393320,4093.70,255724.2709,13675.2672,0.0000,269399.5381,...,124239.575433,143416.7711,162648.2931,181879.815100,201111.337100,220342.859100,236532.106433,249516.100100,259294.840100,265868.326433
3,3,26.335833,75.645000,804.716667,1.401997,3980.56,256324.5263,13708.3632,0.0000,270032.8895,...,124678.935000,143910.4570,163141.9790,182373.501000,201605.023000,220836.545000,237049.069917,250056.341167,259858.358750,266455.122667
4,4,26.752500,76.239166,541.016667,1.403245,3935.55,255829.4703,13681.8864,0.0000,269511.3567,...,124297.067700,143528.5897,162760.1117,181991.633700,201223.155700,220454.677700,236643.925033,249627.918700,259406.658700,265980.145033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,23,26.881667,58.576667,459.025000,1.173069,27895.87,84543.1884,12856.1412,6038.8575,103438.1871,...,42712.123333,50584.1140,58677.2280,68052.857333,77523.692467,85669.334650,92532.461500,98068.848350,102057.371867,103436.640050
780,24,24.530000,59.615833,446.708333,1.190556,3292.69,146626.3049,22339.8000,10503.3825,179469.4874,...,85288.697000,100944.2290,116599.7610,132255.293000,146646.432367,158429.347767,167603.007833,174167.412567,178122.561967,179468.456033
781,25,24.466667,60.382500,455.916667,1.191734,2995.68,146617.0226,22319.9424,10495.5500,179432.5150,...,85055.259400,100710.7914,116366.3234,132021.855400,146445.223283,158260.882883,167467.287150,174064.436083,178052.329683,179430.967950
782,26,25.591667,60.102500,429.241667,1.172986,2620.28,146632.4931,22266.9888,10473.6190,179373.1009,...,85078.171500,100733.7035,116389.2355,132044.767500,146453.898667,158255.837233,167448.520467,174031.948367,178006.120933,179371.038167


In [21]:
X = train.drop(['predicted_weight_g'], axis=1)
y = train['predicted_weight_g']
from sklearn.model_selection import StratifiedKFold, KFold

skf = KFold(n_splits=5, shuffle=True, random_state=404)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

XGB_model = {}

for f in range(5):
    print(f'===================================={f+1}============================================')
    train_idx, val_idx = folds[f]

    x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]

    xgb = XGBRegressor(n_estimators=5000, learning_rate=0.05, subsample=0.5,
                       max_depth=6, gamma=500, reg_lambda=500, colsample_bytree=0.5, random_state=42)
    xgb.fit(x_train, y_train)

    y_pred = xgb.predict(x_val)
    rmse = mean_squared_error(y_val, y_pred)**0.5
    print(f"{f + 1} Fold RMSE = {rmse}")
    XGB_model[f] = xgb
    print(f'================================================================================\n\n')


submit = test[['DAT', 'predicted_weight_g']]
x_test = test.drop(['predicted_weight_g'], axis=1)
for fold in range(5):
    submit['predicted_weight_g'] += XGB_model[fold].predict(x_test)/5
all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
for idx, test_path in enumerate(all_target_list):
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
    submit_df.to_csv(test_path, index=False)

====================================1============================================
1 Fold RMSE = 13.111832882967468


====================================2============================================
2 Fold RMSE = 13.541611509450577


====================================3============================================
3 Fold RMSE = 10.897795076565597


====================================4============================================
4 Fold RMSE = 13.059043829494


====================================5============================================
5 Fold RMSE = 14.043638278961364


